# Bibliotecas

In [78]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# Importando o dataset, renomeando coluna, e split em treino e teste

In [79]:
data = pd.read_csv('Covid Data.csv')
data = data.rename(columns = {'CLASIFFICATION_FINAL': 'CLASSIFICATION_FINAL'})
data1 = data.copy()
data1.loc[data1['DATE_DIED'] == '9999-99-99', 'DIED'] = 2
data1.loc[data1['DATE_DIED'] != '9999-99-99', 'DIED'] = 1
data1.drop(columns = ['DATE_DIED'], inplace = True)

In [80]:
label = 'DIED'
features = list(set(data1.columns).difference({label}))
X = data1[features]
y = data1[label]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

# Pré-Processamento

In [81]:
boolean_features = ['PNEUMONIA', 'PREGNANT', 'DIABETES', 'COPD', 'ASTHMA', 'INMSUPR', 'HIPERTENSION',
                   'CARDIOVASCULAR', 'RENAL_CHRONIC', 'OTHER_DISEASE', 'OBESITY', 'TOBACCO',
                   'INTUBED', 'ICU']

In [82]:
def create_boolean_columns(data, boolean_features):
    '''Given a Pandas DataFrame and a list of string feature names, this function creates
    new boolean columns for each feature in the list. The new columns are 1 if the value
    of the feature is less than 3, and 0 otherwise.'''
    new_data = data.copy()
    for feature in boolean_features:
        new_data.loc[new_data[feature] < 3, f'is_{feature}_defined'] = 1
        new_data.loc[new_data[feature] >= 3, f'is_{feature}_defined'] = 2
    return new_data

def correct_pregnant_for_men(data):
    '''Given a Pandas DataFrame, this function sets the value of the 'PREGNANT' feature to 0
    for all rows where the value of the 'SEX' feature is 2 (corresponding to men).'''
    new_data = data.copy()
    new_data.loc[new_data['SEX'] == 2, 'PREGNANT'] = 0
    return new_data

def mode_imputation(data, pre_imputation_train_data, boolean_features):
    '''Given a Pandas DataFrame, a Pandas DataFrame with the original training data used to
    create the model, and a list of string feature names, this function imputes the mode
    value for each feature in the list for all rows where the value of the feature is 3
    or above (corresponding to missing values).'''
    new_data = data.copy()
    for feature in boolean_features:
        most_common = pre_imputation_train_data[feature].mode()[0]
        new_data.loc[new_data[feature] >= 3, feature] = most_common
    return new_data

def intubed_and_icu_imputation(data):
    '''Given a Pandas DataFrame, this function sets the value of the 'INTUBED' and 'ICU'
    features to 3 (corresponding to missing values) for all rows where the value is 3 or above.'''
    new_data = data.copy()
    more_nan_features = ['INTUBED', 'ICU']
    for feature in more_nan_features:
        new_data.loc[new_data[feature] >= 3, feature] = 3
    return new_data

def covid_degree(data):
    '''takes in a pandas DataFrame and returns a copy of the DataFrame with an added column called 'covid_degree'.
    The 'covid_degree' column is based on the 'CLASSIFICATION_FINAL' column in the input DataFrame. 
    If the value in the 'CLASSIFICATION_FINAL' column is greater than or equal to 4, the corresponding value in the 'covid_degree' column is 0. 
    If the value in the 'CLASSIFICATION_FINAL' column is less than 4, the corresponding value in the 'covid_degree' column 
    is the same as the value in the 'CLASSIFICATION_FINAL' column. 
    The 'CLASSIFICATION_FINAL' column is then dropped from the DataFrame.'''
    new_data = data.copy()
    new_data.loc[new_data['CLASSIFICATION_FINAL'] >= 4, 'covid_degree'] = 0
    new_data.loc[new_data['CLASSIFICATION_FINAL'] < 4, 'covid_degree'] = new_data['CLASSIFICATION_FINAL']
    new_data.drop('CLASSIFICATION_FINAL', axis = 1, inplace = True)
    return new_data

def scale(feature, unscaled_train_feature):
    '''Scales a feature so that its values lie between 0 and 1.'''
    minimum = min(unscaled_train_feature)
    maximum = max(unscaled_train_feature)
    return (feature - minimum)/(maximum - minimum)

def binary_change(data):
    '''This function takes in a pandas DataFrame or a list or numpy array 
    and returns a copy of the DataFrame or array with all 2's replaced with 0's.'''
    new_data = data.copy()
    new_data.loc[new_data == 2] = 0
    return new_data

In [83]:
def pre_processing_pipeline(X_train, X_test, y_train, y_test, boolean_features):
    '''Applies each pre-processing function in the training and test datas.'''
    X_train = create_boolean_columns(X_train, boolean_features)
    X_test = create_boolean_columns(X_test, boolean_features)
    
    X_train = correct_pregnant_for_men(X_train)
    X_test = correct_pregnant_for_men(X_test)
    
    pre_imputation_X_train = X_train.copy()
    X_train = mode_imputation(X_train, pre_imputation_X_train, boolean_features)
    X_test = mode_imputation(X_test, pre_imputation_X_train, boolean_features)
    
    X_train = intubed_and_icu_imputation(X_train)
    X_test = intubed_and_icu_imputation(X_test)
    
    X_train = covid_degree(X_train)
    X_test = covid_degree(X_test)
    
    features = X_train.columns
    unscaled_X_train = X_train.copy()
    
    for feature in features:
        X_train[feature] = scale(X_train[feature], unscaled_X_train[feature])
        X_test[feature] = scale(X_test[feature], unscaled_X_train[feature])
        
    y_train = binary_change(y_train)
    y_test = binary_change(y_test)
    
    return X_train, X_test, y_train, y_test

In [84]:
X_train, X_test, y_train, y_test = pre_processing_pipeline(X_train, X_test, y_train, y_test, boolean_features)

In [85]:
features = X_train.columns

In [86]:
X_train.head()

,ASTHMA,OBESITY,AGE,HIPERTENSION,PREGNANT,OTHER_DISEASE,SEX,RENAL_CHRONIC,INTUBED,COPD,...,is_INMSUPR_defined,is_HIPERTENSION_defined,is_CARDIOVASCULAR_defined,is_RENAL_CHRONIC_defined,is_OTHER_DISEASE_defined,is_OBESITY_defined,is_TOBACCO_defined,is_INTUBED_defined,is_ICU_defined,covid_degree
592908,1.0,0.0,0.239669,1.0,0.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0
184386,1.0,1.0,0.223140,0.0,0.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
1021782,1.0,1.0,0.206612,1.0,0.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
59606,1.0,0.0,0.214876,1.0,1.0,1.0,0.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0
93792,1.0,1.0,0.380165,1.0,1.0,1.0,0.0,1.0,0.5,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [87]:
y_train.head()

592908     0.0
184386     0.0
1021782    0.0
59606      0.0
93792      0.0
Name: DIED, dtype: float64

# Modelo

A função de custo utilizada no modelo de regressão logística é a log loss:

$$Log\;Loss = -\frac{1}{n}\sum_{i = 1}^{n} y\;log(\hat{y}) + (1-y)\;log(1-\hat{y})$$

sendo y a label do dataset de treino (0 ou 1) e $\hat{y}$ o valor da label previsto pelo modelo (algo entre 0 e 1) para dados valores de features. O valor de $\hat{y}$ é obtido pelo uso da função de ativação sigmoid em um modelo linear. Assim, definimos:

$$z = b + \sum_{i = 1}^{m} w_{i} x_{i}$$

$$\hat{y} = sigmoid(z) = sig(z) = \frac{1}{1 + e^{-z}}$$

em que $w_{i}$ é o peso da feature $x_{i}$ no modelo.

Nesse modelo, também aplicaremos a regularização $L_{1}$, responsável por levar os pesos de features pouco importantes (com pesos muito pequenos) a exatamente 0, realizando, assim, uma seleção de features que gera um modelo mais esparso. A função de regularização $L_{1}$ é dada por:

$L_{1}\;regularization\;term = \sum_{i = 1}^{m} |w_{i}|$

Assim, a função que vamos querer minimizar é:

$$f(\theta) = Log\;Loss + L_{1}\;regularization\;term = -\frac{1}{n}\sum_{i = 1}^{n} y_{i}\;log(\hat{y}_{i}) + (1-y_{i})\;log(1-\hat{y}_{i}) + \lambda\sum_{i = 1}^{m} |w_{i}|$$

onde $\lambda$ é a constante de regularização, $\theta = (w_{1}, w_{2}, ..., w_{n}, b)$, sendo $n$ o número de linhas de dados que temos no dataset de treino e $m$ o número de features.

Sabemos que: $\frac{d\hat{y}_{i}}{dz_{i}} = sig(z_{i}) \cdot (1-sig(z_{i}))$ (verifique!)

Assim, podemos calcular as derivadas parciais de $\hat{y}_{i}$: 

$$\frac{\partial{\hat{y}_{i}}}{\partial{w_{k}}} = \frac{d\hat{y}_{i}}{dz_{i}} \cdot \frac{\partial{z_{i}}}{\partial{w_{k}}} = sig(z_{i})\cdot(1-sig(z_{i}))\cdot x_{k}$$

para toda feature $x_{k}$, e:

$$\frac{\partial{\hat{y}_{i}}}{\partial{b}} = \frac{d\hat{y}_{i}}{dz_{i}} \cdot \frac{\partial{z_{i}}}{\partial{b}} = sig(z_{i})\cdot(1-sig(z_{i}))$$

Calculando agora as derivadas parciais de $f(\theta)$:

$$\frac{\partial{f}}{\partial{w_{k}}} = -\frac{1}{n}\sum_{i = 1}^{n}\left[y_{i}\cdot \frac{sig(z_{i})\cdot(1-sig(z_{i}))\cdot (x_{k})_{i}}{\hat{y}_{i}} - (1-y_{i})\cdot\frac{sig(z_{i})\cdot(1-sig(z_{i}))\cdot (x_{k})_{i}}{(1-\hat{y}_{i})}\right] + \lambda \frac{|w_{k}|}{w_{k}}$$

$$\frac{\partial{f}}{\partial{b}} = -\frac{1}{n}\sum_{i = 1}^{n}\left[y_{i}\cdot \frac{sig(z_{i})\cdot(1-sig(z_{i}))}{\hat{y}_{i}} - (1-y_{i})\cdot\frac{sig(z_{i})\cdot(1-sig(z_{i}))}{(1-\hat{y}_{i})}\right]$$

Assim, sendo $\alpha$ a taxa de aprendizado do modelo, os novos valores dos pesos e do viés serão:

$$w_{k}' = w_{k} - \alpha \frac{\partial{f}}{\partial{w_{k}}}$$

$$b' = b - \alpha \frac{\partial{f}}{\partial{b}}$$

Desenvolvendo, temos, por fim:

$$w_{k}' = w_{k} + \frac{\alpha}{n}\sum_{i = 1}^{n}(x_{k})_{i}\cdot\left[y_{i}\cdot {(1-\hat{y}_{i})} - (1-y_{i})\cdot \hat{y}_{i}\right] - \alpha \lambda \frac{|w_{k}|}{w_{k}}$$

$$b' = b + \frac{\alpha}{n}\sum_{i = 1}^{n}\left[y_{i}\cdot {(1-\hat{y}_{i})} - (1-y_{i})\cdot \hat{y}_{i}\right] $$

# Implementação

In [177]:
class Logistic_Regression():
    def __init__(self, X_train, y_train, ws: list,
                 b = 0, alpha = 0.1, lambda_reg = 0.1, random_state = 0):
        '''Constructor for the Logistic_Regression class. It initializes the following attributes:
        - features: a pandas DataFrame containing the training features
        - label: a pandas Series containing the training labels
        - ws: a numpy array containing the weights
        - b: a float representing the bias term
        - alpha: a float representing the learning rate
        - lambda_reg: a float representing the regularization strength
        - rand: a random number generator with a specified seed (random_state)'''
        self.features = X_train
        self.label = y_train
        ws = [float(w) for w in ws]
        self.ws = np.array(ws) # weights
        self.b = b
        self.alpha = alpha
        self.lambda_reg = lambda_reg
        self.rand = np.random.RandomState(random_state)
        
    def print_parameters(self):
        '''Prints the weights and bias term.'''
        for i in range(1, len(self.ws) + 1):
            print(f'w{i} = {self.ws[i - 1]}')
        print (f'b = {self.b}')
        
    def get_parameters(self):
        '''Returns a dictionary containing the weights and bias term. 
        The keys are 'w1', 'w2', etc. for the weights, and 'b' for the bias term.'''
        i_vals = list(range(1, len(self.ws) + 1))
        parameters = {f'w{i}': self.ws[i - 1] for i in i_vals}
        parameters['b'] = self.b
        return parameters
    
    @staticmethod
    def sigmoid(z):
        '''Takes in a float or a numpy array and returns the sigmoid of the input.'''
        return 1/(1 + np.exp(-z))
    
    def predict(self, X):
        '''Takes in a Pandas DataFrame X containing feature values 
        and returns a NumPy array of predicted values of the label using 
        the current values of ws and b.'''
        n = len(X)
        X_copy = X.copy()
        X_copy.reset_index(inplace = True, drop = True)
        X_copy = X.mul(self.ws)
        predictions = X_copy.sum(axis = 1) + self.b
        predictions = Logistic_Regression.sigmoid(predictions)
        return predictions
    
    def get_loss(self, X, y):
        '''Takes in a pandas DataFrame containing the features (X) 
        and a pandas Series (y) containing the labels, 
        and returns the log loss for the given data.'''
        predictions = self.predict(X)
        fst_term = y * np.log(predictions)
        sec_term = (1 - y) * np.log(1 - predictions)
        loss = -np.mean(fst_term + sec_term)
        return loss
    
    def get_l1_term(self):
        '''Gets L1 regularization term for the model.'''
        return np.sum(np.abs(self.ws))
    
    def get_structural_risk(self, X_test, y_test):
        '''Gets structural risk for the model in X_test features and y_test label.'''
        return self.get_loss(X_test, y_test) + self.lambda_reg * self.get_l1_term
    
    def __get_Xy_sample(self, begin_index, end_index):
        '''Private helper function which from X and y starting in the begin_index 
        row and ending in end_index row.'''
        X = self.features.iloc[begin_index:end_index]
        y = self.label.iloc[begin_index:end_index]
        return X, y
    
    def __get_partial_reg_terms(self):
        get_partial_reg_term = np.vectorize(lambda w: 0 if w == 0 else np.abs(w)/w)
        return get_partial_reg_term(self.ws)
    
    def __get_sample_partial_w(self, X, diff, batch_size):
        '''Private helper function which gets the partial derivative of loss with 
        respect to weights for a sample (X).'''
        partial_reg_terms = self.__get_partial_reg_terms()
        partial_w = (-diff / batch_size) @ X + self.lambda_reg * partial_reg_terms
        return partial_w
    
    def __get_sample_partial_b(self, diff, batch_size):
        partial_b = -(1/batch_size) * np.sum(diff)
        return partial_b
    
    def __batch_update_parameters(self, X, diff, batch_size, inexact_batch_size):
        '''Private helper function which updates weights (ws) and bias (b) for a batch.'''
        partial_w = self.__get_sample_partial_w(X, diff, inexact_batch_size)
        partial_b = self.__get_sample_partial_b(diff, inexact_batch_size)
        correction_constant = batch_size/inexact_batch_size
        self.ws -= self.alpha * partial_w * correction_constant
        self.b -= self.alpha * partial_b * correction_constant
        
    def __sgd_update_parameters(self, batch_size: int):
        '''Private helper function that performs one step of stochastic gradient descent on the model's parameters (ws and b). 
        It takes in a single argument batch_size, which is the number of samples to use in the mini-batch for this step of gradient descent. 
        The function first selects mini-batches from the training data, and then performs an update to the model's parameters 
        using the gradient of the mean squared error loss with respect to the parameters for each mini-batch.
        The update is performed using the learning rate alpha.'''
        num_of_data_rows = len(self.label)
        inexact_batch_size = num_of_data_rows % batch_size
        num_of_exact_batches = int(num_of_data_rows/batch_size)
        for exact_batch in range(1, num_of_exact_batches + 1):
            begin_index = (exact_batch - 1) * batch_size
            end_index = exact_batch * batch_size
            X, y = self.__get_Xy_sample(begin_index, end_index)
            predictions = self.predict(X)
            fst_term = y * (1 - predictions)
            sec_term = (1 - y) * predictions
            diff = fst_term - sec_term
            self.__batch_update_parameters(X, diff, batch_size, batch_size)
        if inexact_batch_size != 0:
            begin_index = (num_of_exact_batches) * batch_size + 1
            end_index = num_of_data_rows + 1
            X, y = self.__get_Xy_sample(begin_index, end_index)
            predictions = self.predict(X)
            fst_term = y * (1 - predictions)
            sec_term = (1 - y) * predictions
            diff = fst_term - sec_term
            self.__batch_update_parameters(X, diff, batch_size, inexact_batch_size)
        
    def sgd(self, iterations: int, batch_size: float, print_loss = False): # stochastic gradient descent
        '''Performs stochastic gradient descent for a specified number of iterations.'''
        for i in range(0, iterations):
            self.__sgd_update_parameters(batch_size)
            if print_loss:
                print(f'loss = {self.get_loss(self.features, self.label)}')

In [180]:
ws = list(np.zeros(len(features)))
model = Logistic_Regression(X_train = X_train, y_train = y_train, ws = ws, lambda_reg = 0.01, alpha = 0.5)

In [181]:
model.sgd(iterations = 100, batch_size = 1000000, print_loss = True)

loss = 0.25277087235830625
loss = 0.22969711330010847
loss = 0.21573104893821424
loss = 0.2082657248450225
loss = 0.20437863836128545
loss = 0.20156243154154205
loss = 0.19899296794019258
loss = 0.19654858471517273
loss = 0.1942892097179991
loss = 0.19224464690474152
loss = 0.19021320782163195
loss = 0.18833480205529682
loss = 0.18657770711848887
loss = 0.1850015245541661
loss = 0.18348433172192788
loss = 0.18205280680554464
loss = 0.180638690032133
loss = 0.1793218842584825
loss = 0.17812100312238435
loss = 0.17695845789264486
loss = 0.1758730408237387
loss = 0.1749336055245433
loss = 0.17390631572689388
loss = 0.1730125037319838
loss = 0.1721788218870454
loss = 0.1714142227060946
loss = 0.17066542371856608
loss = 0.1699283174844078
loss = 0.16928594583391746
loss = 0.16863227307370607
loss = 0.16809109725797453
loss = 0.1674717553697401
loss = 0.16695080619905617
loss = 0.16645513279902363
loss = 0.16602351871225504
loss = 0.1653950664826844
loss = 0.16509998020563837
loss = 0.164737

In [182]:
model.predict(X_test)

875680     0.020914
1046906    0.020975
646861     0.022825
704385     0.021049
798051     0.020930
             ...   
986581     0.021062
916423     0.020920
73306      0.023098
565582     0.023277
754044     0.020920
Length: 209715, dtype: float64

In [183]:
threshold = 0.19
print('Contagem dos valores da predição:')
(pd.DataFrame(model.predict(X_test)) > threshold).astype(int).value_counts()

Contagem dos valores da predição:


0    177412
1     32303
dtype: int64

#### Conclusão

Apesar do número reduzido de iterações e da velocidade menor em relação às bibliotecas de Machine Learning já implementadas, conseguimos um resultado válido. Isso mostra que nosso modelo funciona, de fato. 

Não implementaremos as métricas do modelo de regressão logística porque são relativamente simples e o objetivo era conseguir implementar tal modelo do zero com o grande desafio de tratar milhões de dado simultaneamente na função de predição com uma velocidade que permitisse o treinamento do modelo em tempo hábil. A evaluação das métricas pode ser feita utilizando a biblioteca sklearn, conforme foi feito no notebook 01, que trata de feature engineering passo-a-passo e de uma aplicação mais prática do modelo de regressão logística.